In [9]:
import os
import requests

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.llms import HuggingFaceTextGenInference

- https://github.com/langchain-ai/langchain/blob/master/docs/docs/expression_language/cookbook/multiple_chains.ipynb
- https://python.langchain.com/docs/expression_language/cookbook/multiple_chains

In [10]:
hf_server_url = 'env'

server_url = os.environ.get('HUGGINGFACE_INFERENCE_URL', hf_server_url)
server_url

# Server info

In [11]:
response = requests.get(server_url + "/info")
response.json()

{'model_id': 'my',
 'model_device_type': 'cuda',
 'model_pipeline_tag': 'text-generation',
 'max_concurrent_requests': 128,
 'max_best_of': 2,
 'max_stop_sequences': 4,
 'max_input_length': 1024,
 'max_total_tokens': 2048,
 'waiting_served_ratio': 1.2,
 'max_batch_total_tokens': 204176,
 'max_waiting_tokens': 20,
 'max_batch_size': None,
 'validation_workers': 2,
 'docker_label': 'sha-7dbaf9e'}

# tgi llm modell

In [12]:
llm = HuggingFaceTextGenInference(
    inference_server_url=server_url,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.1,
    repetition_penalty=1.03,
    stop_sequences=['\nBot:', '\nUser:'],
    streaming=True,
)

# Single chain

In [13]:
pos_prompt_template = PromptTemplate.from_template(
    "List five positive attributes or benefits of {base_response}. Only the list of positives should be returned."
)
base_model = llm


chain = (
    pos_prompt_template
    | base_model
    | StrOutputParser()
)

# Text examples
skiing_text = "what a feeling it is to go skiing and you always get wonderfully tired."
hockey_text = "when you love Saturday and hockey!"

In [14]:
response = chain.invoke({"base_response": skiing_text})
print(response)



5 positive attributes or benefits of going skiing and getting wonderfully tired:

1. Physical exercise: Skiing provides an excellent full-body workout, helping to improve cardiovascular fitness, strength, and endurance.
2. Stress relief: The physical exertion and fresh mountain air can help reduce stress and promote relaxation.
3. Mental stimulation: Skiing requires focus, balance, and coordination, which can enhance cognitive function and mental agility.
4. Social interaction: Skiing is often enjoyed with friends and family, fostering strong relationships and creating lasting memories.
5. Natural environment: Skiing takes place in beautiful, snow-covered landscapes, offering a chance to connect with nature and appreciate its beauty.


In [15]:
sports = ["Basketball", "Football", "Handball", "Fika"]

for sport in sports:
    response = chain.invoke({"base_response": sport})
    print(response)



5 positive attributes of Basketball:
1. Physical fitness and health benefits
2. Teamwork and social skills development
3. Strategic thinking and problem-solving skills
4. Stress relief and mental well-being
5. Fun and enjoyment


5 positive attributes of Football:
1. Physical fitness and health benefits
2. Teamwork and social skills development
3. Discipline and mental toughness
4. Emotional and psychological well-being
5. Community and cultural bonding


5 positive attributes or benefits of Handball:
1. Physical fitness: Handball is a high-intensity sport that improves cardiovascular endurance, strength, and agility.
2. Teamwork: It fosters teamwork, communication, and cooperation among players.
3. Strategic thinking: Handball requires strategic thinking and quick decision-making skills.
4. Social interaction: It promotes social interaction and camaraderie among teammates and opponents.
5. Mental toughness: Handball helps build mental toughness and resilience through competitive pla

In [16]:
countries = ["United States of America", "France", "Poland", "Cameroon"]

for country in countries:
    response = chain.invoke({"base_response": country})
    print(response)



5 positive attributes or benefits of the United States of America:

1. Economic Power: The United States has one of the largest and most powerful economies in the world, providing numerous opportunities for growth and prosperity.
2. Diversity: The US is a melting pot of cultures, religions, and ethnicities, fostering a rich and vibrant society.
3. Education: The country offers a wide range of educational opportunities, from public schools to prestigious universities, making it a hub for learning and innovation.
4. Freedom and Rights: The US Constitution guarantees certain fundamental rights and freedoms, such as freedom of speech and religion, which are envied by many around the world.
5. Technological Advancements: The United States has been at the forefront of technological advancements, leading to innovations that have transformed industries and improved lives globally.


5 positive attributes or benefits of France:
1. Rich cultural heritage and historical landmarks
2. Delicious c

# Single LLMChain

In [17]:
# Create an LLMChain instance
llm_chain = LLMChain(llm=base_model, prompt=pos_prompt_template, verbose=True)

llm_chain.invoke({"base_response": skiing_text})



> Entering new LLMChain chain...
Prompt after formatting:
List five positive attributes or benefits of what a feeling it is to go skiing and you always get wonderfully tired.. Only the list of positives should be returned.

> Finished chain.


{'base_response': 'what a feeling it is to go skiing and you always get wonderfully tired.',
 'text': '\n\n5 positive attributes or benefits of going skiing and getting wonderfully tired:\n\n1. Physical exercise: Skiing provides an excellent full-body workout, helping to improve cardiovascular fitness, strength, and endurance.\n2. Mental relaxation: The repetitive motion and focus required for skiing can help reduce stress and promote mental relaxation.\n3. Fresh air and nature immersion: Skiing often takes place in beautiful outdoor environments, allowing for fresh air and a connection with nature.\n4. Social interaction: Skiing can be a great way to spend time with friends and family, fostering social connections and creating lasting memories.\n5. Sense of accomplishment: Mastering new skills and overcoming challenges on the slopes can lead to a sense of personal achievement and satisfaction.'}

In [18]:
llm_chain.invoke({"base_response": hockey_text})



> Entering new LLMChain chain...
Prompt after formatting:
List five positive attributes or benefits of when you love Saturday and hockey!. Only the list of positives should be returned.

> Finished chain.


{'base_response': 'when you love Saturday and hockey!',
 'text': "\n\n5 positive attributes or benefits of loving Saturday and hockey:\n\n1. Relaxation and rejuvenation: Saturdays provide a much-needed break from the week's routine, allowing you to unwind and recharge.\n2. Quality time with friends and family: Hockey games on Saturdays can be a great bonding experience, strengthening relationships and creating lasting memories.\n3. Physical activity and health benefits: Participating in hockey or watching it live encourages an active lifestyle, promoting overall well-being.\n4. Entertainment and enjoyment: Watching a thrilling hockey game on Saturday can be a fun and exciting way to spend your free time.\n5. Community and camaraderie: Being part of a hockey community fosters a sense of belonging and shared passion, enhancing social connections."}

In [19]:
for country in countries:
    response = llm_chain.invoke({"base_response": country})
    print(response)



> Entering new LLMChain chain...
Prompt after formatting:
List five positive attributes or benefits of United States of America. Only the list of positives should be returned.

> Finished chain.
{'base_response': 'United States of America', 'text': "\n\nThe United States of America is a diverse and dynamic country with many positive attributes and benefits. Here are five:\n\n1. Economic Powerhouse: The US has one of the largest economies in the world, providing numerous job opportunities and a high standard of living for its citizens.\n2. Cultural Diversity: The US is a melting pot of cultures, offering a rich tapestry of traditions, cuisines, and perspectives that contribute to its vibrant society.\n3. Land of Opportunity: With a strong emphasis on individual freedom and entrepreneurship, the US is often seen as a land of opportunity, where hard work and determination can lead to success.\n4. Education and Research: The US is home to some of the world's top universities and research

# Call same prompt multiple times

In [20]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

In [21]:
model_parser = base_model | StrOutputParser()

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else."
)
prompt1 = ChatPromptTemplate.from_template(
    "generate a color name based on the given attribute, which could describe its intensity, tone, or mood, such as 'vibrant', 'muted', 'warm', or 'cool'. After determining an appropriate color that fits the description of the {attribute}, return only the name of that specific color. Ensure that the response is limited to the color name itself, with no additional explanation or text."
)
prompt1 = ChatPromptTemplate.from_template(
    "Generate a color with the specified attribute, such as 'bright', 'pale', or 'deep'. Return only the name of the resulting {attribute} color and nothing else."
)
prompt1 = ChatPromptTemplate.from_template(
    "Generate a {attribute} color. Return the name of the color and nothing else. Only one color should be returned"
)

prompt1 = ChatPromptTemplate.from_template(
    "Consider an attribute presented to you, which could describe its intensity, tone, or mood, such as 'vibrant', 'muted', 'warm', or 'cool'. Imagine a color that perfectly embodies this specific {attribute}. This color should be strongly associated with and represent the essence of the attribute. Once you have envisioned this color, provide only the name of the color that you believe most accurately captures and represents the given attribute. Return only the color name that best represents the attribute '{attribute}'. Remember to answer nothing else then the name of the color that best represents {attribute}."
)
prompt1 = ChatPromptTemplate.from_template(
    "Consider an attribute presented to you. Imagine a color that perfectly embodies this specific {attribute}. This color should be strongly associated with and represent the essence of the attribute. Once you have envisioned this color, provide only the name of the color that you believe most accurately captures and represents the given attribute. Return that color name. Remember to answer nothing else then the name of the color that best represents {attribute}. The name of the color should be represented in one word."
)
# prompt1 = ChatPromptTemplate.from_template(
#     "Return the color that best represents the attribute '{attribute}'"
# )
prompt1 = ChatPromptTemplate.from_template(
    "# Instruction: What is the name of the color that represents {attribute}? Remember to answer nothing else then the name of the color that best represents {attribute}. \nAnswer: "
)


prompt2 = ChatPromptTemplate.from_template(
    "generate the color when mixing two specified colors, {color1} and {color2}, to see the new color from their combination. Return the name of the color and nothing else:"
)

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser} | itemgetter("color")
)


In [22]:
color_generator.invoke({"attribute": "warm"})

'\n\nThe color that best represents the attribute \'warm\' is "Orange".'

In [23]:
color_generator.invoke({"attribute": "cool"})

'\n\nStep 1: Understand the question\nThe user wants to know the name of a color that represents the attribute "cool".\n\nStep 2: Analyze the attribute\nThe attribute "cool" can be interpreted as something that is calm, refreshing, or associated with cold temperatures.\n\nStep 3: Identify the color\nBased on the analysis, the color that best represents the attribute "cool" is blue. Blue is often associated with calmness, tranquility, and cold temperatures.\n\nStep 4: Provide the answer\nAnswer: Blue'

In [24]:
color_generator.invoke({"attribute": "happiness"})

'\n\nYellow'

In [25]:
color_generator.invoke({"attribute": "sad"})

"\n\nThe name of the color that represents {'attribute': 'sad'} is blue."

In [26]:
color_generator.invoke({"attribute": "love"})

'❤️'

In [34]:
color_generator.invoke({"attribute": "ice cold"})

'\n\nStep 1: Understand the question\nThe user is asking for the name of a color that represents the attribute "ice cold."\n\nStep 2: Analyze the attribute\nThe attribute "ice cold" suggests a feeling of extreme coldness, which is often associated with winter and snow.\n\nStep 3: Identify the color\nBased on the analysis, the color that best represents the attribute "ice cold" is white, as it is commonly associated with snow and winter.\n\nAnswer: White'

In [27]:
color_chain = (
    {"attribute": RunnablePassthrough()}
    | prompt1
    | base_model
    | StrOutputParser()
)
color_chain.invoke({"attribute": "warm"})

"\n\nThe color that best represents the attribute 'warm' is typically red."

In [28]:
two_color_chain = RunnableParallel(
    color1 = {"attribute": itemgetter("attribute1")}
        | color_generator,
    color2 = {"attribute": itemgetter("attribute2")}
        | color_generator
)
combine_color_generator = two_color_chain | prompt2 | {"two_color": model_parser}


In [29]:
two_color_chain.invoke({"attribute1": "warm", "attribute2": "ocean"})

{'color1': '\n\nThe color that best represents the attribute \'warm\' is "Orange".',
 'color2': '\n\nThe name of the color that represents the attribute \'ocean\' is typically referred to as "blue".'}